# Tutorial for using PMO database

We illustrate briefly the main functionality of the package `PMO.jl` dedicated to Polynomial & Moment Optimization problems. The database contains 3 types of data:

- Polynomial Optimization Programs (POP)
- Moment Optimization prograMs (MOM)
- Semi-Definite Programs (SDP)

## Polynomial data

Polynomial optimization problems are of the form 
$$
\begin{array}{rl}
\inf_{x\in \mathbb{R}^n} & f(x)\\ 
 s.t. & g_1(x)\ge 0, \ldots, g_s(x) \geq 0\\
      & h_1(x) = 0, \ldots, h_t(x) =0
\end{array}
$$
where $f, g_1, \ldots, g_s, h_1, ..., h_t \in \mathbb{R}[x_1, \ldots, x_n]$.

 * $f$ is the objective function,
 * $g_1, \ldots, g_s$ are sign constraints,
 * $h_1, ..., h_t$ are equality constraints.

Suppose we have the following polynomial program:
$$
\begin{array}{rl}
\inf_{x,y} & x^2y^2 + x^4 - y^3\\ 
 s.t. & x^2 + \pi y^2 - 2 \leq 0 \\
      & x \in [-1,1] \\
      & 2y^2 - y =0
\end{array}
$$

In [1]:
using PMO, DynamicPolynomials

X = @polyvar x y

f  = x^2*y^2+x^4-y^3
g1 = x^2 + Float64(pi)*y^2 -2
g2 = x

h1 = 2*y^2-y


P  = PMO.polynomial((f,"inf"),
                    (g1,"<=0"),
                    (g2,[-1,1]),
                    (h1,"=0"))

P["doc"]= 
"""
A first polynomial example with two sign constraints and one equality constraint.
"""

"A first polynomial example with two sign constraints and one equality constraint.\n"

Sign constraints  can be of the form `"<=0"` and `">=0"` or an interval `[.,.]`. Notice that `">0"`or `"<0"` are **not** handled.  

The same data `P` could also have been defined as follows:

In [2]:
P  = PMO.data((f,"inf"),
              (g1,"<=0"),
              (g2,[-1,1]),
              (h1,"=0") ; type = "polynomial")

Optimisation model:
  type => polynomial
  variables => ["x", "y"]
  nvar => 2
  constraints => [ x^2 + 3.141592653589793*y^2 - 2.0 <=0, x in [-1, 1], 2*y^2 - y =0 ]
  objective => inf x^4 + x^2*y^2 - y^3
  version => 0.0.1
  uuid => a9c3bdb2-9dd0-11eb-22b7-c5e0ca525f2f


The data contains the following information: 

- the type `P[:type]` specifying if it is polynomial or moment or SDP program
- the variables `P[:variables]` as an array of strings
- the number of variables `P[:nvar]`
- the constraints `P[:constraints]` (optional)
- the objective function `P[:objective]` (optional)
- the version of the data `P[:version]`
- a universally unique identifier `P[:uuid]`

Attributes can be modified, e.g. `P[:version]= "0.0.2"`

New attributes can be added to the data, e.g. `P[:ref] = ["doi1", "doi2"]`.

Attributes can be removed, e.g. `P[:version]= nothing` (but this is not recommended)

This object of type `PMOData` can be printed and saved in json format as follows:

In [3]:
PMO.write(P)
PMO.write("tmp.json",P)

{
  "type": "polynomial",
  "variables": [
    "x",
    "y"
  ],
  "nvar": 2,
  "constraints": [
    {
      "set": "<=0",
      "polynomial": {
        "coeftype": "Float64",
        "terms": [
          [1.0,[2],[1]],
          [3.141592653589793,[2],[2]],
          [-2.0]
        ]
      }
    },
    {
      "set": [
        -1,
        1
      ],
      "polynomial": {
        "coeftype": "Int64",
        "terms": [
          [1,[1],[1]]
        ]
      }
    },
    {
      "set": "=0",
      "polynomial": {
        "coeftype": "Int64",
        "terms": [
          [2,[2],[2]],
          [-1,[1],[2]]
        ]
      }
    }
  ],
  "objective": {
    "set": "inf",
    "polynomial": {
      "coeftype": "Int64",
      "terms": [
        [1,[4],[1]],
        [1,[2,2]],
        [-1,[3],[2]]
      ]
    }
  },
  "version": "0.0.1",
  "uuid": "a9c3bdb2-9dd0-11eb-22b7-c5e0ca525f2f"
}


We read the data saved in the file `tmp.json` and built the corresponding data `Q` of type `PMOData`, which `json` writing is the same as the one of `P`.

In [4]:
Q  = PMO.read("tmp.json")
PMO.write(Q)

{
  "type": "polynomial",
  "variables": [
    "x",
    "y"
  ],
  "nvar": 2,
  "constraints": [
    {
      "set": "<=0",
      "polynomial": {
        "coeftype": "Float64",
        "terms": [
          [1.0,[2],[1]],
          [3.141592653589793,[2],[2]],
          [-2.0]
        ]
      }
    },
    {
      "set": [
        -1,
        1
      ],
      "polynomial": {
        "coeftype": "Int64",
        "terms": [
          [1,[1],[1]]
        ]
      }
    },
    {
      "set": "=0",
      "polynomial": {
        "coeftype": "Int64",
        "terms": [
          [2,[2],[2]],
          [-1,[1],[2]]
        ]
      }
    }
  ],
  "objective": {
    "set": "inf",
    "polynomial": {
      "coeftype": "Int64",
      "terms": [
        [1,[4],[1]],
        [1,[2,2]],
        [-1,[3],[2]]
      ]
    }
  },
  "version": "0.0.1",
  "uuid": "a9c3bdb2-9dd0-11eb-22b7-c5e0ca525f2f"
}


## Moment data

Moment optimization problems are of the form 
$$
\begin{array}{rl}
\inf_{\mu_j\in \mathbb{R}[x_1, \ldots, x_n]^* } & \langle\mu_1|f_1\rangle+\cdots +\langle\mu_\nu|f_\nu\rangle\\ 
 s.t. & \sum_{j=1}^{\nu} g_{1,j} \star \mu_j \succeq 0, \ldots, \sum_{j=1}^{\nu} g_{s,j} \star \mu_j \succeq 0\\
      & \sum_{j=1}^{\nu} h_{1,j} \star \mu_j = 0, \ldots, \sum_{j=1}^{\nu} h_{t,j} \star \mu_j =0\\
      & \sum_{j=1}^{\nu} \langle\mu_j|l_{1,j}\rangle+ l_{1,\nu+1}\ge 0, \ldots, \sum_{j=1}^{\nu}\langle\mu_j|l_{u,j}\rangle+l_{u,\nu+1}\ge 0\\
      & \sum_{j=1}^{\nu} \langle\mu_j|m_{1,j}\rangle + m_{1,\nu+1}= 0, \ldots,  \sum_{j=1}^{\nu}\langle\mu_j|m_{v,j}\rangle = 0
\end{array}
$$
where
  * $\mu_1, \ldots, \mu_\nu$ are moment sequences, i.e. elements of the dual $\mathbb{R}[x_1, \ldots, x_n]^*$,
  * $f, g_{i,j}, h_{i,j}, l_{i,j}, m_{i,j}\in \mathbb{R}[x_1, \ldots, x_n]$.

The constraints $\sum_{j=1}^{\nu} \langle\mu_i|l_{i,j}\rangle+l_{i,\nu+1}\ge 0$, $\sum_{j=1}^{\nu} \langle\mu_j|m_{i,j}\rangle +m_{i,\nu+1}= 0$ are respectively mass sign constraints and mass equality constraints (specified with `">=0 *"` and `"=0 *`). They can have non-zero constant terms $l_{i,\nu+1},m_{i,\nu+1}$ or not. 

The product $\star$ is defined as follows: for $p\in \mathbb{R}[x_1, \ldots, x_n], \mu \in \mathbb{R}[x_1, \ldots, x_n]^*$, $p\star \mu : q \mapsto \langle p\star \mu|q\rangle:= \langle\mu| p q\rangle$.  

Here is an example:
$$
\begin{array}{rl}
\inf_{\mu_1,\mu_2} & \langle \mu_1 | x^2y^2 + x^4 - y^3 \rangle + \langle \mu_2 | x y  \rangle \\ 
 s.t. & (x^2 + \pi y^2 - 2) \star \mu_1 \succeq 0 \\
      & \langle  \mu_2 | x \rangle -1 \geq 0 \\
      & \langle  \mu_1 | 2y^2 - y \rangle  + \langle \mu_2 | x^2 + 2.1 x y^2 \rangle = 0 
\end{array}
$$

In [5]:
using DynamicPolynomials, PMO
@polyvar x y
f1 = x^2*y^2+x^4-y^3
f2 = x*y

g1 = x^2 + Float64(pi)*y^2 -2
g2 = x

h1 = 2*y^2-y
h2 = x^2+y*2.1*x*y

F  = PMO.moment(([f1,f2],"inf"),
                ([g1,0],">=0"),
                ([0,g2, -1], ">=0 *"),
                ([h1, h2], "=0 *")
                )

Optimisation model:
  type => moment
  variables => ["x", "y"]
  nvar => 2
  nms => 2
  constraints => [ Polynomial{true,Float64}[x² + 3.141592653589793y² - 2.0, 0.0] * >=0, Polynomial{true,Int64}[0, x, -1] * >=0 *, Polynomial{true,Float64}[2.0y² - y, 2.1xy² + x²] * =0 * ]
  objective => inf Polynomial{true,Int64}[x⁴ + x²y² - y³, xy]
  version => 0.0.1
  uuid => acd88e4a-9dd0-11eb-120e-79957d943e8c


In [6]:
PMO.write("tmp.json",F)
G  = PMO.read("tmp.json")
PMO.write(G)

{
  "type": "moment",
  "variables": [
    "x",
    "y"
  ],
  "nvar": 2,
  "nms": 2,
  "constraints": [
    {
      "set": ">=0",
      "moments": {
        "coeftype": "Float64",
        "terms": [
          [1.0,1,[2],[1]],
          [3.141592653589793,1,[2],[2]],
          [-2.0,1]
        ]
      }
    },
    {
      "set": ">=0 *",
      "moments": {
        "coeftype": "Int64",
        "terms": [
          [1,2,[1],[1]],
          [-1,3]
        ]
      }
    },
    {
      "set": "=0 *",
      "moments": {
        "coeftype": "Float64",
        "terms": [
          [2.0,1,[2],[2]],
          [-1.0,1,[1],[2]],
          [2.1,2,[1,2]],
          [1.0,2,[2],[1]]
        ]
      }
    }
  ],
  "objective": {
    "set": "inf",
    "moments": {
      "coeftype": "Int64",
      "terms": [
        [1,1,[4],[1]],
        [1,1,[2,2]],
        [-1,1,[3],[2]],
        [1,2,[1,1]]
      ]
    }
  },
  "version": "0.0.1",
  "uuid": "acd88e4a-9dd0-11eb-120e-79957d943e8c"
}


## SDP data


SDP optimization problems are of the form

$$
\begin{array}{rl}
\inf_{x\in \mathbb{R}^n} & f^T x \\ 
 s.t. & \sum_{j=1}^{n} A_{1,j} x_j + A_{1,0} \succeq 0, \ldots, \sum_{j=1}^{n} A_{s,j} x_j + A_{s,0} \succeq 0\\
      & \sum_{j=1}^{n} c_{1,1}\cdot x + d_{1,1}\ge 0, \ldots, \sum_{j=1}^{n} c_{1,u}\cdot x + d_{1,u}\ge 0\\
      & \sum_{j=1}^{n} c_{0,1}\cdot x + d_{0,1} = 0, \ldots, \sum_{j=1}^{n} c_{0,v} \cdot x + d_{0,v} = 0
\end{array}
$$
where $f, c_{i,j} \in \mathbb{R}^n$, $d_{i,j}\in \mathbb{R}$, $A_{i,j} \in S^{n_i}$ are symmetric matrices of size $n_i$.

Let us consider the following problem 
$$
\begin{array}{rl}
\inf_{x_1,x_2,x_3} &  x_1 + 2 x_2 + 3 x_3 \\ 
 s.t. & \begin{pmatrix}
2 & -1 & 0\\
-1 & 2 & 0 \\
0 & 0 & 2
\end{pmatrix} x_1 + 
\begin{pmatrix}
2 & 0 & -1\\
0 & 2 & 0 \\
-1 & 0 & 2
\end{pmatrix} x_3 \succeq 0 , \\
& \begin{pmatrix}
1 & 0 \\
0 & -1 
\end{pmatrix} x_1 + 
\begin{pmatrix}
0 & 3 \\
3 & 0 
\end{pmatrix} x_2 + 
\begin{pmatrix}
0 & -1 \\
-1 & 2 
\end{pmatrix}  \succeq 0, \\
      & 1.1 x_1 + 2 x_2  -4 = 0, \quad -1.2 x_2 + 3 x_3 -1 \leq 0
\end{array}
$$
It can be built as follows:

In [7]:
using PMO, LinearAlgebra

LMI1 = [Symmetric([2 -1 0; 0 2 0; 0 0 2]),
        0,
        Symmetric([2 0 -1; 0 2 0; 0 0 2])
       ]

LMI2 = [Symmetric([1 0; 0 -1]),
        Symmetric([0 3; 3 0 ]),
        0,
        Symmetric([0 -1; -1 2])
       ]

M  = PMO.sdp(([1,2,3], "inf"),
             (LMI1,">=0"),
             (LMI2,">=0"),
             ([1.1,2,0,-4], "=0"),
             ([0,-1.2,3,-1],"<=0"),
             )
M[:name] = "My first example"
M[:doc]  = "Two linear matrix inequalities, one linear scalar equality and one linear scalar inequality."

"Two linear matrix inequalities, one linear scalar equality and one linear scalar inequality."

In [8]:
PMO.write("tmp.json",M)
N  = PMO.read("tmp.json")
PMO.write(N)

{
  "type": "sdp",
  "nvar": 3,
  "objective": [1,2,3],
  "constraints": {
    "nlmi": 2,
    "msizes": [3,2],
    "lmi_symat": [
      [2.0,1,1,1,1],
      [-1.0,1,1,2,1],
      [2.0,1,1,2,2],
      [2.0,1,1,3,3],
      [2.0,3,1,1,1],
      [2.0,3,1,2,2],
      [-1.0,3,1,3,1],
      [2.0,3,1,3,3],
      [1.0,1,2,1,1],
      [-1.0,1,2,2,2],
      [3.0,2,2,2,1],
      [-1.0,0,2,2,1],
      [2.0,0,2,2,2]
    ],
    "nlsi": 2,
    "lsi_mat": [
      [1.1,1,1],
      [2.0,1,2],
      [1.2,2,2],
      [-3.0,2,3]
    ],
    "lsi_vec": [4.0,-1.0],
    "lsi_op": [0,1]
  },
  "version": "0.0.1",
  "uuid": "adb50140-9dd0-11eb-1f9e-f12a0ed41040",
  "name": "My first example",
  "doc": "Two linear matrix inequalities, one linear scalar equality and one linear scalar inequality."
}


In [9]:
LMI1 = [Symmetric([2 1 0; 0 1 0; 0 0 0]),
        Symmetric([0 0 0; 0 1 1; 0 0 1]),
        [[0,0,1]],
        -Symmetric([0 0 0; 0 0 1; 0 0 0])]

F  = PMO.sdp(([0,0,1], "inf"),
             (LMI1, ">=0"),
             ([1, 1, 0 , -1], "=0"),
             ([1, 0, 0 ], ">=0"),
             ([0, 1, 0 ], ">=0")
             )
F[:name] = "A second example"
F[:doc] =
    """
    one LMI with one rank-1 matrix, 3 linear scalar constraints
    """

"one LMI with one rank-1 matrix, 3 linear scalar constraints\n"

Here in `LMI1`, the third matrix is a low rank matrix, represented as a vector of vectors $u_i$ corresponding to $\sum_i u_i\otimes u_i$.

In [10]:
PMO.write("tmp.json",M)
N = PMO.read("tmp.json")
PMO.write(N)

{
  "type": "sdp",
  "nvar": 3,
  "objective": [1,2,3],
  "constraints": {
    "nlmi": 2,
    "msizes": [3,2],
    "lmi_symat": [
      [2.0,1,1,1,1],
      [-1.0,1,1,2,1],
      [2.0,1,1,2,2],
      [2.0,1,1,3,3],
      [2.0,3,1,1,1],
      [2.0,3,1,2,2],
      [-1.0,3,1,3,1],
      [2.0,3,1,3,3],
      [1.0,1,2,1,1],
      [-1.0,1,2,2,2],
      [3.0,2,2,2,1],
      [-1.0,0,2,2,1],
      [2.0,0,2,2,2]
    ],
    "nlsi": 2,
    "lsi_mat": [
      [1.1,1,1],
      [2.0,1,2],
      [1.2,2,2],
      [-3.0,2,3]
    ],
    "lsi_vec": [4.0,-1.0],
    "lsi_op": [0,1]
  },
  "version": "0.0.1",
  "uuid": "adb50140-9dd0-11eb-1f9e-f12a0ed41040",
  "name": "My first example",
  "doc": "Two linear matrix inequalities, one linear scalar equality and one linear scalar inequality."
}


## Using the database

The PMO data base is loaded as follows:

In [11]:
using PMO
t = PMO.table()

PMO.DataBase(Table with 30 rows, 3 columns:
Columns:
#  colname  type
──────────────────
1  uuid     String
2  name     String
3  url      String)

It is a table of triplets `(uuid , name, url)` of a unique identifier `uuid`, a `name` containing tags to recover easily the data and the `url` of the file containing the data. This table only contains references to the PMO data. These data files are available in the local folder `$HOME/.julia/PMO/data`. 

The data files of the `git` server are visible at https://github.com/PolynomialMomentOptimization/data/tree/master/json. 

The registery of the database on the `git` server is at https://github.com/PolynomialMomentOptimization/data/blob/master/registries/index-pmo.csv.

## Selecting data

Data can be selected from their `:name` attribute, by regular expressions or matching strings.

In [12]:
t2 = select(t,r"[Mm]otz")
t3 = select(t,"Motzkin")

PMO.DataBase(Table with 3 rows, 3 columns:
Columns:
#  colname  type
──────────────────
1  uuid     String
2  name     String
3  url      String)

This gives new tables `t2, t3` of triplets `(uuid,name,url)` (Here with 3 rows). 

To select one column of such a table, one can do the following:

In [13]:
select(t2, :name)

3-element WeakRefStrings.StringArray{String,1}:
 "Motzkin"            
 "Motzkin homogeneous"
 "Motzkin bounded"    

We can recover the i$^{th}$ entry of the table as follows:

In [14]:
t2[1]

Optimisation model:
  type => polynomial
  variables => Any["x", "y"]
  nvar => 2
  objective => inf x^4*y^2 + x^2*y^4 - 3*x^2*y^2 + 1
  name => Motzkin
  version => 0.0.2
  author => Bernard Mourrain
  uuid => cb46cb04-83db-11ea-0b43-87115082d076
  doc => Celebrate Motzkin polynomial, which is a non-negative polynomial, but not a Sum of Squares



Data can also be recovered from regular expressions matching their names:

In [15]:
t2[r"Motz."]

3-element Array{Any,1}:
 Optimisation model:
  type => polynomial
  variables => Any["x", "y"]
  nvar => 2
  objective => inf x^4*y^2 + x^2*y^4 - 3*x^2*y^2 + 1
  name => Motzkin
  version => 0.0.2
  author => Bernard Mourrain
  uuid => cb46cb04-83db-11ea-0b43-87115082d076
  doc => Celebrate Motzkin polynomial, which is a non-negative polynomial, but not a Sum of Squares

                          
 Optimisation model:
  type => polynomial
  variables => Any["x", "y", "z"]
  nvar => 3
  constraints => [ x^2 + y^2 + z^2 - 1 =0 ]
  objective => inf x^4*y^2 + x^2*y^4 - 3*x^2*y^2*z^2 + z^6
  version => 0.0.2
  uuid => e46b1efe-0a05-11eb-198e-0912f9f7d742
  name => Motzkin homogeneous
  doc => Homogeneous Motzkin polynomial on the unit sphere

  author => Bernard Mourrain

 Optimisation model:
  type => polynomial
  variables => Any["x", "y"]
  nvar => 2
  constraints => [ -x^2 - y^2 + 2 >=0 ]
  objective => inf x^4*y^2 + x^2*y^4 - 3*x^2*y^2 + 1
  version => 0.0.2
  uuid => dab9e7be-19fa-11e

or equivalently the elements, which name contains "Motz":

In [16]:
m = t3["Motz"]

3-element Array{PMO.Data,1}:
 Optimisation model:
  type => polynomial
  variables => Any["x", "y"]
  nvar => 2
  objective => inf x^4*y^2 + x^2*y^4 - 3*x^2*y^2 + 1
  name => Motzkin
  version => 0.0.2
  author => Bernard Mourrain
  uuid => cb46cb04-83db-11ea-0b43-87115082d076
  doc => Celebrate Motzkin polynomial, which is a non-negative polynomial, but not a Sum of Squares

                          
 Optimisation model:
  type => polynomial
  variables => Any["x", "y", "z"]
  nvar => 3
  constraints => [ x^2 + y^2 + z^2 - 1 =0 ]
  objective => inf x^4*y^2 + x^2*y^4 - 3*x^2*y^2*z^2 + z^6
  version => 0.0.2
  uuid => e46b1efe-0a05-11eb-198e-0912f9f7d742
  name => Motzkin homogeneous
  doc => Homogeneous Motzkin polynomial on the unit sphere

  author => Bernard Mourrain

 Optimisation model:
  type => polynomial
  variables => Any["x", "y"]
  nvar => 2
  constraints => [ -x^2 - y^2 + 2 >=0 ]
  objective => inf x^4*y^2 + x^2*y^4 - 3*x^2*y^2 + 1
  version => 0.0.2
  uuid => dab9e7be-19f

Data can be selected and retrieved via test functions:

In [17]:
hasprop = function(x) return (PMO.getdata(x[:url])[:nvar] >= 3) end # test if the data has a number of variables >= 3
nwl = t[hasprop];
nwt = select(t,hasprop)

PMO.DataBase(Table with 26 rows, 3 columns:
Columns:
#  colname  type
──────────────────
1  uuid     String
2  name     String
3  url      String)

Iterators can also be used on tables: `[x[:name] for x in t]`


## Updating the database

The database can be updated from the git server as follows:

In [18]:
update(t)

┌ Info: PMO update data
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:45


Table with 30 rows, 3 columns:
Columns:
#  colname  type
──────────────────
1  uuid     String
2  name     String
3  url      String

#### Adding new data

New data can be added to the database via `git`and `push` commands. For that to work properly, you have to get a read/write access to the  `github` project [https://github.com/PolynomialMomentOptimization/data](https://github.com/PolynomialMomentOptimization/data). If not, please create an account on https://github.com/ and ask to join the project. 

We illustrate the process on a simple example. First we create the data:

In [19]:
using PMO, DynamicPolynomials
X = @polyvar x y
motz = x^4*y^2 + x^2*y^4 + 1 - 3*x^2*y^2

Motz = PMO.data((motz,"inf"), (2-x^2-y^2, ">=0"))
Motz[:name] = "Motzkin bounded"

"Motzkin bounded"

Now we push it to the database with the file name `motzkin_bounded`:

In [20]:
t = PMO.table()
push(t, Motz, file="motzkin_bounded")

┌ Warning: data[:author] or data[:name] not defined; data not pushed to database
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:248


The name `motzkin_bounded` specifies the file used to store the data. 

Here we get a warning because the attribute `:author` is not defined in Motz. So we define it and retry:

In [21]:
Motz[:author] = "Joe Test"
push(t, Motz, file="motzkin_bounded")

┌ Info: PMO update data
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:45
┌ Info: PMO data motzkin_bounded.json already exists
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:121
┌ Info: PMO add data motzkin_bounded.1.json
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:139
┌ Info: PMO update registery
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:195
┌ Info: PMO update data
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:45
┌ Info: PMO update table
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:258


We see that the file name `motzkin_bounded` is already used so that the data is stored as the file `motzkin_bounded.1.json` in the folder `json`of the data project `PMO.PMO_GIT_DATA_URL`. The index table of the registry folder is updated with a line per data. See
  - https://github.com/PolynomialMomentOptimization/data/tree/master/json
  - https://github.com/PolynomialMomentOptimization/data/blob/master/registries/index-pmo.csv

A local version of the data bases and index files are updated; `git` is used to commit and push the changes on this `github` project.

Data can be removed from the database as follows:

In [22]:
PMO.rm(t, Motz)

┌ Info: PMO remove data motzkin_bounded.1.json
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:112


#### Modifying a data 

Here, it is showing how to modify an element and update of the database:

In [23]:
F = t[2]
F[:version] = "0.0.2"

"0.0.2"

To update of the database element of the table `t`:

In [24]:
push(t,F)

┌ Warning: same data; no commit and push
└ @ PMO /Users/mourrain/Julia/PMO.jl/src/register.jl:266


The data would be modified locally and on the `git` server. But here because it is the same text, there is a warning saying that no modification is commited by `git`.

In [25]:
t[2]

Optimisation model:
  type => polynomial
  variables => Any["x", "y"]
  nvar => 2
  objective => inf x^4*y^2 + x^2*y^4 - 3*x^2*y^2 + 1
  name => Motzkin
  version => 0.0.2
  author => Bernard Mourrain
  uuid => cb46cb04-83db-11ea-0b43-87115082d076
  doc => Celebrate Motzkin polynomial, which is a non-negative polynomial, but not a Sum of Squares



#### Managing tables

Tables can be saved and loaded from files, as illustrated in the following example.

Suppose you want to work on all data related to `Motzkin`. First, you can select them from the PMO database and build a new database `tm`:

In [26]:
using PMO
t = PMO.table()
tm = select(t,"Motz")

PMO.DataBase(Table with 3 rows, 3 columns:
Columns:
#  colname  type
──────────────────
1  uuid     String
2  name     String
3  url      String)

Next you can save it in a file`motztable.csv` for futur use:

In [27]:
PMO.write("motztable.csv",tm)

Once saved in the file local `motztable.csv`, the table can be reloaded without loading the complete PMO database, as follows:

In [28]:
tr = PMO.table("motztable.csv")

PMO.DataBase(Table with 3 rows, 3 columns:
Columns:
#  colname  type
──────────────────
1  uuid     String
2  name     String
3  url      String)

## Solving optimization problems

The PMO data can be transformed with `vec` into an array of pairs of polynomials and constraints that can be easily used in other functions: 

In [29]:
P = t[r"Motz.*bound"][1]
vec(P)

2-element Array{Any,1}:
 (x⁴y² + x²y⁴ - 3x²y² + 1, "inf")
 (-x² - y² + 2, ">=0")           

Here is an illustration with the packages `MomentTools`, `MosekTools`, using the functions `vec` and `variables`:

In [30]:
using MomentTools, MosekTools
#optimizer = Mosek.Optimizer    #for verbose execution of Mosek solver
using JuMP; optimizer = optimizer_with_attributes(Mosek.Optimizer, "QUIET"=>true)  # for silent execution.
v, M = optimize(vec(P),variables(P), 3, optimizer)     # Moment relaxation at order 3
get_minimizers(M) 

2×4 Array{Float64,2}:
 -0.999949  -0.999949   0.999949  0.999949
 -0.999949   0.999949  -0.999949  0.999949

Here is another illustration with the package `TSSOS`:

In [31]:
using TSSOS
tssos_first(first.(vec(P)), variables(P), 3)

***************************TSSOS***************************
TSSOS is launching...
Starting to compute the block structure...
------------------------------------------------------
The sizes of PSD blocks:
[3, 1]
[3, 1]
------------------------------------------------------
Obtained the block structure in 1.510642412 seconds. The maximal size of blocks is 3.
Assembling the SDP...
SDP assembling time: 0.440671189 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 10              
  Cones                  : 0               
  Scalar variables       : 5               
  Matrix variables       : 4               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator t

(5.30280704779727e-8, nothing, TSSOS.cpop_data(2, 0, 1, 0, PolyVar{true}[x, y], Polynomial{true,Int64}[x⁴y² + x²y⁴ - 3x²y² + 1, -x² - y² + 2], Polynomial{true,Int64}[], Array{UInt8}(undef,2,0), Array{UInt8,2}[[0x04 0x02 0x02 0x00; 0x02 0x04 0x02 0x00], [0x02 0x00 0x00; 0x00 0x02 0x00]], Array{Float64,1}[[1.0, 1.0, -3.0, 1.0], [-1.0, -1.0, 2.0]], Array{UInt8,2}[[0x00 0x01 … 0x01 0x00; 0x00 0x00 … 0x02 0x03], [0x00 0x01 … 0x01 0x00; 0x00 0x00 … 0x01 0x02]], UInt8[0x00 0x02 … 0x00 0x02; 0x00 0x00 … 0x06 0x02], [3, 1], [3, 1], Array{Array{UInt16,1},1}[[[0x0001, 0x0004, 0x0006], [0x0002, 0x0007, 0x0009], [0x0003, 0x0008, 0x000a], [0x0005]], [[0x0001, 0x0004, 0x0006], [0x0002], [0x0003], [0x0005]]], UInt16[0x0004, 0x0004], Array{UInt16,1}[[0x0003, 0x0003, 0x0003, 0x0001], [0x0003, 0x0001, 0x0001, 0x0001]], "Mosek", 0.0001, 1))